# Introduction to Machine Learning
## Home Assignment 1

In [4]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import random

In [5]:
# We get the data 
_URL = 'http://www.di.ens.fr/appstat/spring-2020/project/data.zip'

path_to_zip = tf.keras.utils.get_file('data.zip', origin=_URL, extract=True)

data_path = os.path.join(os.path.dirname(path_to_zip), 'data')

training_data_path = os.path.join(data_path, 'train')

test_data_path = os.path.join(data_path, 'test')

In [6]:
# We define the constants

number_of_letters = 6000
number_of_samples_per_letter = 2000
pixel_height = 28
pixel_width = 28
batch_size = 50
shape = (pixel_height*pixel_width)
seed = 54
# We define a flow of images: 

def generate (dir_path): 
    training_data = []
    training_labels = []
    for item in os.listdir(dir_path):
        tmp = os.path.join(dir_path,item)
        if os.path.isdir(tmp):
            d, l = generate (tmp)
            training_data = training_data + d
            training_labels = training_labels + l
        elif tmp[-3:] == "png":
            im = Image.open(tmp)
            training_data.append(np.reshape(np.asarray(im), shape))
            if(dir_path[-1:] == 'A'):
                training_labels.append(1)
            else: 
                training_labels.append(-1)
    return training_data, training_labels
            
def make (dir_path): 
    d, l = generate(dir_path)
    data = np.array(d)/255
    random.Random(seed).shuffle(data)
    label = np.reshape(np.array(l), (np.shape(l)[0],1))
    random.Random(seed).shuffle(label)
    return data, label



training_data, training_labels = make(training_data_path)
test_data, test_labels = make(test_data_path)

print(np.shape(training_data))
print(np.shape(training_labels))
    


(6000, 784)
(6000, 1)


1. Formalize the problem by defining the input space $X$ , the output space $Y$ and the training data set. What are their dimension?

Having preprocessed the data we are going to use the images as vectors of pixels with size $n = pixel\_width*pixel\_height$. Thus: \
$X$ is $[0, 1]^n$ and $Y$ is $\{-1, 1\}$ and $D = (X \times Y) $



2. a) What are the empirical risk (training error) and the true risk associated with the 0-1 loss? Why is it complicated to minimize the empirical risk in this case

The empirical risk for the 0-1 loss is : $$R_d(f) = \frac{1}{d} \sum_{i = 1}^{d} \mathbb{1}_{f(X_i) \neq Y_i}$$ and the true risk is : $$R(f) = \mathbb{E}\{\mathbb{1}_{f(X) \neq Y} | D \}$$
The empirical risk here is of the 0-1 loss function is non-convex and discontinuous, thus (sub)gradient methods cannot be applied. Any iterative method would be exponential in the input size thus the choice of the 0-1 loss would be inadequate.

2. b) Why should we use the test data to assess the performance ? 

Test data should be from the same statistical distribution as the training data, and should not have been fed to the learning model before (otherwise there is no point given the model already adjusted to that data before)

2. c) Recall the definition of the optimization problems associated with the linear least square regression andthe linear logistic regression.

We start firstly with linear regression: 
$$f_n\in argmin_{f \in \mathbb F}R_n(f) := argmin_{f \in \mathbb F}\frac{1}{n}\sum_{i = 1}^{n}l(x_i, y_i)$$
with $l$ being the loss function associated to the problem. 
In linear regression, we usually choose the square loss, and as the name states we choose a linear model: $f(x) = \theta^T x$. Thus the linear regression problem becomes: 
$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n}\frac{1}{n}\sum_{i = 1}^{n}( \theta^T x_i - y_i)^2$$
That can simply be rewritten to: 
$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n} \frac{1}{n} || Y - X\theta ||_2^2 $$
with $X \in \mathbb R^{d*n}$ being the design matrix having at the row $i$ $x_i^T$, and $Y \in \mathbb R^d$ being the vector of outputs with $y_i$ at the position $i$

For logistic regression we have: 
$$f_n\in argmin_{f \in \mathbb F}R_n(f) := argmin_{f \in \mathbb F}\frac{1}{n}\sum_{i = 1}^{n}l(x_i, y_i)$$
with $l$ being the loss function associated to the problem. 
In logistic regression, we try to maximize the log-likelihood of an estimator and we end up with a minimization problem using the logistic loss, we choose a linear model: $f(x) = \theta^T x$. Thus the logistic regression problem becomes: 

$$min_{\theta \in \mathbb R^n} R_n(\theta) := min_{\theta \in \mathbb R^n}\frac{1}{n}\sum_{i = 1}^{n}log(1 + e^{−y_i \theta^T x_i}))$$


In [1]:
#we define here some utility functions to compute the regression sum and the gradient

def logistic_regression_loss (theta, x, y): 
    print(np.shape(1/np.shape(y)[0] * np.sum(np.log(1+np.exp(-y*x@theta)), axis=0)))
    return 1/np.shape(y)[0] * np.sum(np.log(1+np.exp(-y*x@theta)), axis=0)

def logistic_gradient (theta, x, y):
    k = -y*x
    print(np.shape(k))
    print(np.shape(theta))
    e = np.exp(k@theta)
    print(np.shape(e))
    u_ = k*e
    print("bro")
    print(np.shape(1/np.shape(y)[0] * np.sum(u_/(1+e), axis=0)))
    return np.reshape(1/np.shape(y)[0] * np.sum(u_/(1+e), axis=0), np.shape(theta)) 


def square_loss (theta, x, y): 
    return 1/np.shape(y)[0] * np.sum((x@theta - y)**2, axis=0)

def square_gradient (theta, x, y):
    return np.reshape(1/np.shape(y)[0]*2*np.sum(x, axis=0)*square_loss(theta, x, y), np.shape(theta))           
    
def sigmoid(t):
    return 1/(1 + np.exp(-t))       

def logistic_classify(theta, data): 
    return 2*(sigmoid(data@theta)>0.5) - 1

def square_classify(theta, data): 
    return 2*(theta@data > 0) - 1


In [59]:
#here we define the gradient descent and the stochastic gradient descent with backtracking line search (to get an optimal learning rate)


epsilon = 10**-4
eta = 0.0000001

def backtracking_line_search (f, x, delta_x, grd_x, alpha= 0.25, beta=0.89):
    t = 1
    while (f(x+t*delta_x) >= f(x) + alptha*t*np.transpose(grd_x)@delta_x):
        t *= beta
    return t

def gradient_descent (f, grd, starting_point, eta = eta, epsilon = epsilon, learning_rate_function = backtracking_line_search):
    theta = starting_point
    prev_theta = starting_point
    r = f(theta, training_data, training_labels)
    print("bro")
    while True:
        prev_theta = theta
        f_theta, delta_theta = r, -grd(theta, training_data, training_labels)
        theta += eta*delta_theta#learning_rate_function(lambda x: regression_sum(x, train_data_gen, f), theta, delta_theta, -delta_theta)*
        r = f(theta, training_data, training_labels)
        print(r - f_theta)
        if (np.absolute(r - f_theta) < epsilon):
            break;
    return theta


def stochastic_gradient_descent (f, grd, starting_point, epsilon=epsilon, learning_rate_function = backtracking_line_search) :
    theta = starting_point
    prev_theta = starting_point
    y = 0
    for x_, y_ in training_data_gen:
        x, y = adjust(x_, y_)
        while True:
            prev_theta = theta
            f_theta, delta_theta = r, -grd(theta, x, y)
            theta += delta_theta#learning_rate_function(lambda t: f(t, x, y), theta, delta_theta, -delta_theta)*
            r = f(theta, x, y)
            print(r - f_theta)
            if (np.absolute(r - f_theta) < epsilon):
                break;
    return theta
    

theta = gradient_descent(square_loss, square_gradient, np.random.uniform(0, 1, (28*28, 1)))
accuracy = 0
for x, y in zip(test_data, test_labels):
    accuracy += sum(logistic_classify(theta, x) == y) 
    
print(accuracy)

bro
[-564.53612918]
[-551.77079822]
[-539.3881613]
[-527.37396319]
[-515.71458191]
[-504.39699613]
[-493.40875445]
[-482.73794649]
[-472.37317564]
[-462.30353335]
[-452.51857489]
[-443.0082965]
[-433.76311379]
[-424.77384135]
[-416.0316735]
[-407.52816606]
[-399.25521917]
[-391.20506097]
[-383.37023222]
[-375.7435717]
[-368.3182024]
[-361.08751844]
[-354.04517267]
[-347.18506491]
[-340.50133078]
[-333.98833116]
[-327.64064215]
[-321.4530455]
[-315.42051961]
[-309.53823087]
[-303.80152558]
[-298.20592209]
[-292.74710349]
[-287.42091052]
[-282.22333494]
[-277.15051313]
[-272.19872006]
[-267.3643635]
[-262.64397854]
[-258.03422234]
[-253.53186913]
[-249.13380549]
[-244.83702575]
[-240.63862772]
[-236.5358085]
[-232.52586058]
[-228.60616804]
[-224.77420297]
[-221.02752203]
[-217.36376315]
[-213.78064243]
[-210.27595108]
[-206.84755261]
[-203.49338001]
[-200.21143322]
[-196.99977653]
[-193.85653621]
[-190.77989821]
[-187.76810594]
[-184.81945816]
[-181.93230696]
[-179.1050558]
[-176.3361576

[-3.72984108]
[-3.71387635]
[-3.6980024]
[-3.68221857]
[-3.66652422]
[-3.65091873]
[-3.63540147]
[-3.6199718]
[-3.60462912]
[-3.58937281]
[-3.57420226]
[-3.55911688]
[-3.54411605]
[-3.5291992]
[-3.51436573]
[-3.49961506]
[-3.48494662]
[-3.47035982]
[-3.45585411]
[-3.44142892]
[-3.4270837]
[-3.41281787]
[-3.39863091]
[-3.38452226]
[-3.37049139]
[-3.35653775]
[-3.34266082]
[-3.32886007]
[-3.31513498]
[-3.30148502]
[-3.28790969]
[-3.27440848]
[-3.26098087]
[-3.24762637]
[-3.23434448]
[-3.22113471]
[-3.20799657]
[-3.19492957]
[-3.18193323]
[-3.16900708]
[-3.15615063]
[-3.14336343]
[-3.13064499]
[-3.11799487]
[-3.10541261]
[-3.09289774]
[-3.08044982]
[-3.0680684]
[-3.05575304]
[-3.04350329]
[-3.03131871]
[-3.01919889]
[-3.00714338]
[-2.99515175]
[-2.9832236]
[-2.97135849]
[-2.959556]
[-2.94781574]
[-2.93613728]
[-2.92452021]
[-2.91296415]
[-2.90146868]
[-2.89003341]
[-2.87865794]
[-2.86734189]
[-2.85608486]
[-2.84488647]
[-2.83374635]
[-2.8226641]
[-2.81163936]
[-2.80067175]
[-2.7897609]
[-

[-0.56796873]
[-0.56662432]
[-0.56528398]
[-0.56394769]
[-0.56261544]
[-0.56128721]
[-0.55996299]
[-0.55864276]
[-0.55732651]
[-0.55601422]
[-0.55470588]
[-0.55340146]
[-0.55210097]
[-0.55080437]
[-0.54951167]
[-0.54822283]
[-0.54693786]
[-0.54565672]
[-0.54437941]
[-0.54310592]
[-0.54183623]
[-0.54057032]
[-0.53930818]
[-0.5380498]
[-0.53679516]
[-0.53554425]
[-0.53429705]
[-0.53305355]
[-0.53181374]
[-0.5305776]
[-0.52934511]
[-0.52811628]
[-0.52689107]
[-0.52566948]
[-0.5244515]
[-0.5232371]
[-0.52202628]
[-0.52081903]
[-0.51961532]
[-0.51841515]
[-0.51721851]
[-0.51602538]
[-0.51483574]
[-0.51364959]
[-0.51246691]
[-0.51128769]
[-0.51011191]
[-0.50893957]
[-0.50777064]
[-0.50660513]
[-0.50544301]
[-0.50428427]
[-0.5031289]
[-0.50197689]
[-0.50082822]
[-0.49968289]
[-0.49854087]
[-0.49740217]
[-0.49626676]
[-0.49513463]
[-0.49400578]
[-0.49288019]
[-0.49175784]
[-0.49063873]
[-0.48952284]
[-0.48841017]
[-0.48730069]
[-0.4861944]
[-0.4850913]
[-0.48399135]
[-0.48289456]
[-0.48180091]

[-0.16408476]
[-0.16377346]
[-0.16346276]
[-0.16315265]
[-0.16284312]
[-0.16253418]
[-0.16222583]
[-0.16191806]
[-0.16161087]
[-0.16130427]
[-0.16099824]
[-0.16069279]
[-0.16038791]
[-0.16008361]
[-0.15977988]
[-0.15947673]
[-0.15917414]
[-0.15887212]
[-0.15857066]
[-0.15826977]
[-0.15796945]
[-0.15766968]
[-0.15737048]
[-0.15707183]
[-0.15677374]
[-0.15647621]
[-0.15617923]
[-0.15588281]
[-0.15558693]
[-0.1552916]
[-0.15499683]
[-0.1547026]
[-0.15440891]
[-0.15411577]
[-0.15382317]
[-0.15353111]
[-0.15323959]
[-0.1529486]
[-0.15265816]
[-0.15236825]
[-0.15207887]
[-0.15179002]
[-0.15150171]
[-0.15121392]
[-0.15092666]
[-0.15063993]
[-0.15035372]
[-0.15006803]
[-0.14978287]
[-0.14949823]
[-0.14921411]
[-0.1489305]
[-0.14864741]
[-0.14836484]
[-0.14808278]
[-0.14780123]
[-0.14752019]
[-0.14723966]
[-0.14695964]
[-0.14668013]
[-0.14640112]
[-0.14612261]
[-0.14584461]
[-0.14556711]
[-0.14529011]
[-0.14501361]
[-0.1447376]
[-0.14446209]
[-0.14418708]
[-0.14391256]
[-0.14363853]
[-0.1433649

[-0.04532317]
[-0.04518689]
[-0.04505072]
[-0.04491467]
[-0.04477874]
[-0.04464292]
[-0.04450721]
[-0.04437162]
[-0.04423614]
[-0.04410077]
[-0.04396551]
[-0.04383037]
[-0.04369534]
[-0.04356042]
[-0.04342561]
[-0.04329091]
[-0.04315633]
[-0.04302185]
[-0.04288748]
[-0.04275322]
[-0.04261907]
[-0.04248502]
[-0.04235109]
[-0.04221726]
[-0.04208354]
[-0.04194993]
[-0.04181642]
[-0.04168302]
[-0.04154972]
[-0.04141653]
[-0.04128345]
[-0.04115047]
[-0.04101759]
[-0.04088482]
[-0.04075215]
[-0.04061958]
[-0.04048712]
[-0.04035476]
[-0.0402225]
[-0.04009034]
[-0.03995828]
[-0.03982633]
[-0.03969447]
[-0.03956272]
[-0.03943106]
[-0.03929951]
[-0.03916805]
[-0.03903669]
[-0.03890543]
[-0.03877427]
[-0.03864321]
[-0.03851224]
[-0.03838137]
[-0.0382506]
[-0.03811992]
[-0.03798934]
[-0.03785886]
[-0.03772847]
[-0.03759817]
[-0.03746797]
[-0.03733786]
[-0.03720785]
[-0.03707793]
[-0.03694811]
[-0.03681837]
[-0.03668873]
[-0.03655918]
[-0.03642972]
[-0.03630036]
[-0.03617108]
[-0.0360419]
[-0.03591

In [ ]:
def k_nn_one_example (k, x, y, example, distance_function):
    n = int(cross_validation_percentage*number_of_letters)
    x_training = x[:n]
    distances = distance_function(x_training, example)
    labels = y[np.argsort(x_training)[:k]]
    return 2*(np.sum(labels) > 0) -1

def KNN (k, data, labels, unlabeled_data, distance_function):
    np.apply_along_axis(k_nn_one_example, 1, unlabeled_data, k, )